In [2]:
import numpy as np
import pandas as pd
import networkx as nx
#from stellargraph.data import BiasedRandomWalk
#from stellargraph import StellarGraph
from gensim.models import Word2Vec
import csv

In [3]:
#Get train.csv data
df = pd.read_fwf('train.csv', header=None)
df = df[0].str.split(',', expand=True) 

In [17]:
#Get ingredient id
df2 = pd.read_fwf('node_ingredient.csv', header=None)
idToNode = {}
for index , row in df2.iterrows():
    idToNode[index] = row[0]


In [18]:
len(idToNode)

6714

In [8]:
#Create dict of dict where the first key is ingredient id the value is another dict 
#with key are nodes he shares receipt with and value the number of receipt they have in common
ingredientAdjancy = {}
ingredientNbR = {}
for index , row in df.iterrows():
    for i in row.values:
        if i is not None and i.isdigit() and int(i) not in ingredientAdjancy.keys():
            ingredientAdjancy[int(i)] = {}
        for y in row.values:
            if i is not None and  y is not None and i.isdigit() and y.isdigit():
                if i != y:
                    if int(y) not in ingredientAdjancy[int(i)].keys():
                        ingredientAdjancy[int(i)][int(y)] = 1
                    else:
                        ingredientAdjancy[int(i)][int(y)] += 1
        if i is not None and i.isdigit():
            if int(i) not in ingredientNbR.keys():
                ingredientNbR[int(i)] = 1
            else:
                ingredientNbR[int(i)] += 1


In [6]:
len(ingredientAdjancy)

5860

In [13]:
for k,v in ingredientAdjancy.items():
    print(k,v)
    break

2813 {3146: 146, 3229: 24, 3885: 40, 4379: 17, 4390: 10, 5250: 18, 5456: 30, 6187: 758, 392: 1, 937: 2279, 1476: 323, 2172: 29, 2351: 179, 3350: 4, 3554: 43, 3857: 74, 3978: 207, 5249: 94, 5648: 397, 145: 3, 163: 10, 203: 29, 628: 80, 1198: 148, 1807: 39, 1955: 21, 5377: 784, 5459: 76, 6352: 62, 826: 4, 2420: 2, 2533: 21, 26: 1, 399: 11, 530: 203, 791: 17, 1308: 224, 1920: 19, 3483: 10, 4060: 21, 4506: 45, 4672: 92, 5816: 22, 6370: 1, 6386: 17, 56: 7, 154: 90, 2122: 465, 2966: 5, 5731: 8, 552: 82, 1112: 52, 1396: 127, 1454: 87, 1679: 73, 2615: 6, 2809: 129, 2890: 2, 3653: 106, 3886: 28, 3965: 49, 4201: 8, 4563: 39, 4799: 234, 5030: 13, 5882: 127, 6126: 100, 6711: 52, 281: 164, 308: 75, 1137: 120, 2414: 66, 3429: 12, 3447: 72, 4373: 99, 4543: 14, 4632: 92, 4844: 29, 5119: 185, 5585: 45, 5884: 276, 1495: 67, 1953: 98, 2945: 513, 4013: 23, 4162: 42, 5163: 1, 170: 32, 1903: 10, 2651: 82, 3371: 70, 4038: 196, 4616: 5, 5654: 52, 5966: 2, 1354: 25, 1661: 42, 3003: 9, 3029: 20, 5457: 9, 5536: 

In [19]:
import csv
fh = open('edgeListWeight.csv','w')
writer = csv.writer(fh)
for i in ingredientAdjancy.keys():
    for y in ingredientAdjancy[i].keys():
        t = str(i)+'\t'+str(y)+'\t'+str(ingredientAdjancy[i][y])
        writer.writerow([t])
fh.close()

In [20]:
import csv
fh = open('edgeList.csv','w')
writer = csv.writer(fh)
for i in ingredientAdjancy.keys():
    for y in ingredientAdjancy[i].keys():
        t = str(i)+'\t'+str(y)
        writer.writerow([t])
fh.close()

In [14]:
Gw = nx.read_edgelist('edgeListWeight.csv', nodetype=int, data=(('weight',float),))
G = nx.read_edgelist('edgeList.csv', nodetype=int, data=(('weight',float),))

In [15]:
print(len(G.nodes()))
print(len((G.edges())))
nodes = list(G.nodes())


5860
353396


Some ingredient are not part of the graph (have no edge)

Use Node2vec -> first need to generate weighted random walk. Then use Word2Vec to generate the embedding.
Generate random walktake quite some time, I already generate 4 of them using different paramters that should adjust the balance between sutrucural and local importance. This might be improved by trying different set of hyperparameter. 

In [19]:
#Use StellarGraph library to generate the weighted random walk
GwS = StellarGraph.from_networkx(Gw)

NameError: name 'StellarGraph' is not defined

In [ ]:
rw = BiasedRandomWalk(GwS)

In [ ]:
weighted_walks = rw.run(
        nodes=G.nodes(), 
        length=100,    # maximum length of a random walk
        n=10,          # number of random walks per root node
        p=1,         # Defines (unormalised) probability, 1/p, of returning to source node
        q=100,         # Defines (unormalised) probability, 1/q, for moving away from source node
        weighted=True, #for weighted random walks
        seed=42        # random seed fixed for reproducibility
    )

In [ ]:
str(weighted_walks[1])

In [ ]:
#Save the random walk
import csv

fh = open('NodeRandomWalkp1q100.csv','w')
writer = csv.writer(fh)
for i in range(5860):
    for y in range(10):
        t = str(GwS.nodes()[i])+" "+str(weighted_walks[i*10+y])
        writer.writerow([t])
fh.close()

In [28]:
#Import random walk
import ast
rp1q10 = pd.read_fwf('RW/WRandomWalkp1q2.csv', header=None, delimiter= " ")

rp1q10_list = []
for index, value in rp1q10.iterrows():
    t = False
    s = ""
    for i in value[0]:
        if i == "[":
            t = True
        if i == '"' and t:
            t = False
        if t:
            s+=i
    if s[len(s)-1] != "]":
        s+="]"
    rp1q10_list.append(ast.literal_eval(s)) 
  

In [29]:
from gensim.test.utils import common_texts
from gensim.models import Word2Vec

embNod2V = Word2Vec(sentences=rp1q10_list, vector_size=64, window=3, min_count=0, sg=1, workers=1, epochs=1)

In [30]:
len(embNod2V.wv)

5865

In [16]:
from numpy import dot
from numpy.linalg import norm
m=0
test = 6000
for i in Gw.nodes():
    if i != test:
        a = embNod2V.wv[test]
        b = embNod2V.wv[i]
        cos_sim = dot(a, b)/(norm(a)*norm(b))
        if cos_sim > m:
            vm = i
            m = cos_sim
print(m,vm)
print(idToNode[test]," is most similar to: ",idToNode[vm])

NameError: name 'embNod2V' is not defined

In [52]:
fh = open('Embedding/Embp1q100.csv','w')
writer = csv.writer(fh)
for i in Gw.nodes():
    writer.writerow([i,embNod2V.wv[i]])
fh.close()

Second embedding method (not weighted)

In [70]:
from sklearn.decomposition import  TruncatedSVD

A = nx.adjacency_matrix(G) 

svd = TruncatedSVD(n_components=128)
X = svd.fit_transform(A);

Save different embedding size (You can find 32-64-128 in /embedding)

In [71]:
import csv

fh = open('Embedding/SVD128.csv','w')
writer = csv.writer(fh)

y = 0
for i in G.nodes():
    writer.writerow([i,X[y]]) 
    y += 1
    
fh.close()

In [74]:
test = 6
m = 0

for i in range(5860):
    if i != test:
        a = X[test]
        b = X[i]
        cos_sim = dot(a, b)/(norm(a)*norm(b))
        if cos_sim > m:
            vm = i
            m = cos_sim
print(m,vm)
print(idToNode[list(G.nodes())[test]]," is most similar to: ",idToNode[list(G.nodes())[vm]])

0.8458904945495505 264
black olives  is most similar to:  olives
